Imports

In [ ]:
import trackintel as ti
import glob
import pandas as pd
import geopandas as gpd
import os
import random

from trackintel.preprocessing import generate_locations, generate_staypoints, generate_triplegs, merge_staypoints

User parameters

In [ ]:
timezone = 'Asia/Shanghai'

data_path = 'Geolife Trajectories 1.3/Data/'

# Set dropout threshold (should match min staypoint duration)
dropout_threshold = pd.Timedelta(minutes=10)
# Set time difference threshold between dropout/staypoint start here
diff_threshold = pd.Timedelta(minutes=1)

Generate initial staypoints

In [ ]:
agents = os.listdir(data_path)
print(len(agents))
print(agents[0])

In [ ]:
# Read a single plt file for an agent
def extract_day(file_path):
    day = pd.read_csv(file_path, names=['latitude', 'longitude', 'zero', 'altitude', 'days', 'date', 'time'], skiprows=6)
    day['tracked_at'] = pd.to_datetime(day['date'] + ' ' + day['time']).dt.tz_localize(timezone)
    day = day.drop(columns=['zero', 'altitude', 'days', 'date', 'time'])

    user_id = file_path[file_path.index('Data')+5:file_path.index('Data')+8]
    day['user_id'] = user_id

    return day

In [ ]:
# Extract raw trajectory
def get_full_traj(agent_path):
    agent_trajs = glob.glob(agent_path+'Trajectory/*.plt')
    agent_traj = pd.concat([extract_day(x) for x in agent_trajs], ignore_index=True)
    agent_traj = agent_traj.sort_values(by=['tracked_at']).reset_index(drop=True)

    # Convert to geo dataframe
    agent_traj = gpd.GeoDataFrame(agent_traj, geometry=gpd.points_from_xy(agent_traj['longitude'], agent_traj['latitude'], crs="EPSG:4326"))

    return agent_traj

In [ ]:
# Because the Geolife data is often much larger and more complex than the T-drive data, we will
# employ a few more tools of the trackintel library to generate high quality initial staypoints

def get_ti_sps(path, timezone='Asia/Shanghai'):
    df = get_full_traj(path)
    if len(df) > 1:
        # WHAT THIS DOES: will return pfs as the original df but with assigned staypoint id (or NA) to each point
        #                 generates initial sps
        # Currently using simple parameters
        pfs, sps = generate_staypoints(
            df,
            dist_threshold=100, # Min dist between staypoints, in meters
            time_threshold=pd.Timedelta(minutes=10), # Min duration to create a staypoint
            gap_threshold=pd.Timedelta(minutes=25), # Max gap time to still mark something as a staypoint
            include_last=True, # Makes sure we include the last one if the user ends there
        )

        # I believe this is the only place we should need this, since we may return no staypoints
        if len(sps) == 0:
            return

        # WHAT THIS DOES: Generates triplegs (which we'll need later) by just looking between sps
        #                 returns pfs as initial pfs but with assigned tripleg id (or NA) to each point. Now, every point should be at a sp or tripleg
        # Taking same gap threshold as in generate_staypoints
        pfs, tpls = generate_triplegs(pfs, gap_threshold=25)

        # WHAT THIS DOES: Adds a location id to each sp so we can make merging staypoints easier
        #                 Could also return locations but we ignore it with underscore
        # Taking default parameters
        sps, _ = generate_locations(sps)

        # WHAT THIS DOES: Merges staypoints that are at the same location, consecutive, and within some time gap of each other
        # Setting max time gap to same as gap threshold; agg is necessary to include geometry, and takes last geometry of merged staypoints
        sps = merge_staypoints(sps, tpls, max_time_gap=pd.Timedelta(minutes=25), agg={"geometry":"last"})
    
        if len(sps) > 0:
            user_id=sps['user_id'].iloc[0]
            sps.to_csv('geolife_sps/user_'+str(user_id)+'.csv')

In [ ]:
for agent in agents:
    print(agent)
    get_ti_sps(data_path+agent)

Generate trajectories with dropouts and staypoints based on these

In [ ]:
# I want to create on average 8 dropouts. Thus, probability is 8 / len(df). This number is variable
def get_dirty_ti_sps(path, avg_num_dropouts=8):
    df = get_full_traj(path)
    if len(df) == 0:
        return
    
    # Create on average 8 dropouts
    dropoutlength = pd.Timedelta(minutes=15) # Can also change dropout time if desired
    dropouts = []
    i = 0
    while i < len(df):
        if random.random() > avg_num_dropouts / len(df):
            i += 1
            continue

        j = i
        while j < len(df) and (df['tracked_at'].iloc[j] - df['tracked_at'].iloc[i]) < dropoutlength:
            j += 1
        
        dropouts.append([i, j])

        i = j
    
    all_dropouts = []
    for dropout in dropouts:
        all_dropouts.extend(list(range(dropout[0], dropout[1])))
    
    # Write indices of dropouts to a text file
    with open('geolife_dropout_indices/user_'+str(df['user_id'].iloc[0])+'.txt', 'w') as f:
        for line in all_dropouts:
            f.write(f"{line}\n")

    # Create a list of start times we dropped for that agent
    lines = [df['tracked_at'].iloc[x[0]] for x in dropouts]
    with open('geolife_dropouts/user_'+str(df['user_id'].iloc[0])+'.txt', 'w') as f:
        for line in lines:
            f.write(f"{line}\n")

    df = df.drop(all_dropouts).reset_index(drop=True)

    # We have to make this check again after adding dropouts
    if len(df) == 0:
        return

    # Rerun trackintel on the trajectory with dropouts
    pfs, sps = generate_staypoints(
        df,
        dist_threshold=100, # Min dist between staypoints, in meters
        time_threshold=pd.Timedelta(minutes=10), # Min duration to create a staypoint
        gap_threshold=pd.Timedelta(minutes=25), # Max gap time to still mark something as a staypoint
        include_last=True, # Makes sure we include the last one if the user ends there
    )
    
    if len(sps) == 0:
        return

    pfs, tpls = generate_triplegs(pfs, gap_threshold=25)

    sps, _ = generate_locations(sps)

    sps = merge_staypoints(sps, tpls, max_time_gap=pd.Timedelta(minutes=25), agg={"geometry":"last"})

    # Write to file
    if len(sps) > 0:
        user_id=sps['user_id'].iloc[0]
        sps.to_csv('geolife_noisy_sps/user_'+str(user_id)+'.csv')

In [ ]:
for agent in agents:
    print(agent)
    get_dirty_ti_sps(data_path+agent)

In [ ]:
# Add a flag for whether the staypoint is spurious
def add_is_spurious(path):
    noisy_sps = ti.read_staypoints_csv(path, index_col=None, geom_col='geometry')
    user_id = noisy_sps['user_id'].iloc[0]

    with open("geolife_dropouts/user_"+str(user_id).zfill(3)+".txt", "r") as file:
        dropouts = [line.strip() for line in file]
    dropouts = pd.Series(dropouts)
    dropouts = pd.to_datetime(dropouts)

    noisy_sps['is_spurious'] = False
    for i in dropouts:
        abs_diff = abs(noisy_sps['started_at'] - i)
        abs_diff = abs_diff <= diff_threshold
        noisy_sps['is_spurious'] = noisy_sps['is_spurious'] | abs_diff
    
    noisy_sps.to_csv(path)

In [ ]:
noised_files = glob.glob('geolife_noisy_sps/*.csv')
for nf in noised_files:
    add_is_spurious(nf)

Now apply filter

In [ ]:
def get_user(agent_id):
    path = data_path + str(agent_id).zfill(3)
    df = get_full_traj(path)
    df['tracked_at']=df['tracked_at'].dt.tz_convert(timezone)

    return df

def get_noisy_user(agent_id):
    path = data_path + str(agent_id).zfill(3)
    df = get_full_traj(path)
    df['tracked_at']=df['tracked_at'].dt.tz_convert(timezone)

    with open('geolife_dropout_indices/user_'+str(agent_id).zfill(3)+'.txt') as file:
        all_dropouts = [int(line.strip()) for line in file]
    
    df = df.drop(all_dropouts).reset_index(drop=True)

    return df

In [ ]:
def filter_dropouts(agent_id):

    # Identify dropouts over dropout threshold
    traj = get_noisy_user(agent_id)
    traj['t_delta'] = traj['tracked_at'].diff(1)
    traj['t_delta'] = traj['t_delta'].shift(-1)

    dropouts = traj[traj['t_delta'] >= dropout_threshold].reset_index(drop=True)
    timezone = 'UTC+08:00'
    dropouts['tracked_at']=dropouts['tracked_at'].dt.tz_convert(timezone)

    noisy_sps = ti.read_staypoints_csv('geolife_noisy_sps/user_'+str(agent_id).zfill(3)+'.csv', index_col=None, geom_col='geometry')

    # This function call identifies the spurious staypoints
    merged = pd.merge_asof(
        noisy_sps,
        dropouts,
        left_on="started_at",
        right_on="tracked_at",
        tolerance=diff_threshold,
        direction="forward"
    )

    # And here we remove these staypoints
    indices_to_drop = noisy_sps.index[merged["tracked_at"].notna()]
    non_dropout_sps = noisy_sps.drop(indices_to_drop).reset_index(drop=True)

    file_path = 'geolife_sps/user_'+str(agent_id).zfill(3)+'.csv'
    if os.path.exists(file_path):
        sps = ti.read_staypoints_csv(file_path, index_col=None, geom_col='geometry')
    else:
        sps = pd.DataFrame() # Should just be able to create an empty df I think

    # We return:    Length of original staypoints, 
    #               length of noised staypoints, 
    #               length of spurious noised staypoints,
    #               length of filtered staypoints
    #               length of filtered spurious staypoints
    
    to_return = [agent_id,
                 len(sps), 
                 len(noisy_sps), 
                 len(noisy_sps[noisy_sps['is_spurious'] == True]), 
                 len(non_dropout_sps), 
                 len(non_dropout_sps[non_dropout_sps['is_spurious'] == True])]
    return to_return

Check results

In [ ]:
results = pd.DataFrame(columns=['user_id', 'num_sps', 'num_noised_sps', 'num_spurious_sps', 'num_filtered_sps', 'num_spurious_filtered_sps'])

users = glob.glob('geolife_noisy_sps/*.csv')
users = [int(user[user.index('user') + 5 : user.index('.')]) for user in users]

for user in users:
    results.loc[len(results)] = filter_dropouts(user)

In [ ]:
# Can specify an output location
results.to_csv('geolife_results.csv')